In [1]:
import numpy as np
import pandas as pd
import random
import itertools
import matplotlib.pyplot as plt
import plotly.express as px
import glob
import spacy
import sklearn
from collections import defaultdict, Counter
from bisect import bisect_left
import matplotlib.pyplot as plt
import matplotlib
import io
import base64
import pymp
from multiprocessing import Pool, Manager, cpu_count
from IPython.display import display_html
from itertools import chain, cycle
import lightgbm as lgb
import sys
sys.path.append('../../../../utils')
sys.path.append('..')
import gezi
from gezi import tqdm
from src.eval import *
from src.util import *
from src import config
from src.visualize import *
from src.rewards import *
pd.set_option('display.float_format', lambda x: '%.02f' % x)

In [2]:
# https://www.kaggle.com/chasembowers/sequence-postprocessing-v2-67-lb/notebook#Sequence-Datasets

In [3]:
root = '../input/feedback-prize-2021'

In [4]:
df = pd.read_feather(f'{root}/train_en.fea')

In [5]:
MAX_SEQ_LEN = {}
df['len'] = df.end - df.start
max_lens = df.groupby('para_type')['len'].quantile(.995)
for i in range(len(ALL_CLASSES)):
  MAX_SEQ_LEN[id2dis[i]] = int(max_lens[i])
MAX_SEQ_LEN

{'Nothing': 132,
 'Claim': 55,
 'Evidence': 299,
 'Position': 59,
 'Concluding Statement': 191,
 'Lead': 197,
 'Counterclaim': 95,
 'Rebuttal': 116}

In [6]:
proba_thresh = {
    "Lead": 0.7,
    "Position": 0.55,
    "Evidence": 0.65,
    "Claim": 0.55,
    "Concluding Statement": 0.7,
    "Counterclaim": 0.5,
    "Rebuttal": 0.55,
    "Nothing": 0.5,
}
# TODO 自动学习这个阈值

In [7]:
d = pd.read_feather(f'{root}/para_label.fea')

In [8]:
gts = {}
for row in tqdm(d.itertuples(), total=len(d)):
  gts[row.id] = decode_label_all(row.start, row.para_types)

  0%|          | 0/15594 [00:00<?, ?it/s]

In [9]:
folds = {}
for row in tqdm(df.itertuples(), total=len(df)):
  folds[row.id] = row.kfold

  0%|          | 0/175160 [00:00<?, ?it/s]

In [10]:
np.random.seed(12345)

In [11]:
subfolds = {}
for row in tqdm(df.itertuples(), total=len(df)):
  subfolds[row.id] = np.random.randint(5)

  0%|          | 0/175160 [00:00<?, ?it/s]

In [12]:
gts['A8445CABFECE']

{0: [[0, 3]],
 1: [[15, 36]],
 2: [[36, 177]],
 3: [[3, 15]],
 4: [[177, 212]],
 5: [],
 6: [],
 7: []}

In [13]:
def para_dataset(info, disc_type=None, start_thre=0.1, start_thre2=0.99, fold=None, subfold=None, infer=False):
  total = len(info['id'])
  fes = []
  scores = []
  matches = []
  for i in tqdm(range(total)):
    x = {}
    for key in info:
      x[key] = info[key][i]
    id = x['id']
    gt = gts[id]
    fe = {}
    fe['id'] = id
    fe['index'] = i
    fe['fold'] = folds[id]
    if fold is not None and fe['fold'] != fold:
      continue
    fe['subfold'] = subfolds[id]
    if subfold is not None and fe['subfold'] != subfold:
      continue
    fe['num_words'] = x['num_words']
    fe['seps'] = (x['start_probs'][:,1] > 0.5).sum()
    fe['sep_ratio'] = fe['seps'] / fe['num_words']
    num_words = x['num_words']
    for j in range(num_words):
      if j > 0 and x['start_probs'][j][1] < start_thre and x['preds'][j] == x['preds'][j - 1]:
        continue
      probs = np.zeros_like(x['probs'][j])
      probs += x['probs'][j]
      fe['start'] = j
      fe['start_ratio'] = (j + 1) / num_words
      fe['start_probs'] = x['probs'][j]
      fe['start_sep_prob'] = x['start_probs'][j][1] if j > 0 else 1.
      preds = {k: 0 for k in range(NUM_CLASSES)}
      fe['start_cls'] = x['preds'][j]
      fe['max_start_prob'] = probs.max()
      preds[x['preds'][j]] += 1
      fe['pre_cls'] = -1 if j == 0 else x['preds'][j - 1]
      fe['pre_max_prob'] = 1 if j == 0 else x['probs'][j - 1].max()
      sep_count = 0
      for k in range(j + 1, num_words):
        probs += x['probs'][k]
        fe['para_len'] = k + 1 - j
        fe['para_len_ratio'] = (k + 1 - j) / num_words
        preds[x['preds'][k]] += 1
        if k + 1 < num_words and x['start_probs'][k + 1][1] < start_thre and x['preds'][k] == x['preds'][k + 1]:
          continue
        if k + 1 < num_words and x['start_probs'][k + 1][1] > 0.5:
          sep_count += 1
        fe['end'] = k + 1
        fe['end_ratio'] = (k + 1) / num_words
        fe['end_probs'] = x['probs'][k]
        fe['max_end_prob'] = x['probs'][k + 1].max() if k + 1 < num_words else 1
        end_cls = np.argmax(x['probs'][k])
        fe['end_cls'] = end_cls
        fe['end_sep_prob'] = x['start_probs'][k + 1][1] if k + 1 < num_words else 1.
        fe['sep_add_prob'] = (fe['start_sep_prob'] + fe['end_sep_prob']) / 2.
        fe['sep_mul_prob'] = (fe['start_sep_prob'] * fe['end_sep_prob']) ** 0.5
        fe['num_classes'] = len([k for k in range(NUM_CLASSES) if preds[k] > 0])
        mean_probs = gezi.softmax(probs)
        fe['mean_probs'] = mean_probs
        fe['max_prob'] = mean_probs.max()
        top_classes = np.argsort(-mean_probs,axis=0)[:2]
        fe['top_class'] = top_classes[0]
        fe['top_class2'] = top_classes[1]
        fe['next_cls'] = -1 if k + 1  == num_words else x['preds'][k + 1]
        # top_classes = top_classes[:1]
        if fe['para_len'] > 1 and fe['num_classes'] < 4:
          for cls in top_classes:
            if disc_type != None and cls != disc_type:
              continue
            if cls and fe['para_len'] >  MAX_SEQ_LEN[id2dis[cls]]:
              continue
            if fe['mean_probs'][cls] < proba_thresh[id2dis[cls]]:
              continue
            fe['mean_prob'] = fe['mean_probs'][cls]
            fe['start_prob'] = x['probs'][j][cls]
            fe['end_prob'] = x['probs'][k][cls]
            fe['class_ratio'] = (x['preds'][j:k+1] == cls).sum() / fe['para_len']
            # fe['class_max_prob'] = x['probs'][j:k+1][cls].max()
            # fe['class_min_prob'] = x['probs'][j:k+1][cls].min()
            fe['cls'] = cls
            fe['is_top_class'] = int(cls == fe['top_class'])
            fe['is_top_class2'] = int(cls == fe['top_class2'])
            if not infer:
              fe['score'] = best_match(gt[cls], [j, k + 1])
              fe['match'] = calc_match(gt[cls], [[j, k + 1]])
              # if not fe['match'] and np.random.randint(3) != 0:
              #   continue
              scores.append(fe['score'])
              matches.append(fe['match'])
            fes.append(fe.copy())
        if k + 1 < num_words and x['start_probs'][k + 1][1] > start_thre2:
          break
        if sep_count > 2:  
          break
  if not infer:
    ic(np.mean(scores), np.mean(matches))
  d = pd.DataFrame(fes)
  return d

In [14]:
info = gezi.load('../working/offline/44/valid.pkl')

In [15]:
nw = 5
ds = Manager().dict()
with pymp.Parallel(nw) as p:
  for fold in p.range(5):
    fold_ = fold
    ds[fold_] = para_dataset(info, fold=fold)

  0%|          | 0/15594 [00:00<?, ?it/s]

ic| <-ipythoninput-13-6f9f2c390751>:98 in para_dataset()
    np.mean(scores): 0.38091122614339173
    np.mean(matches): 0.402760131598067
icipython| <-input-13-6f9f2c390751>:98 in para_dataset()
    np.mean(scores): 0.3816458008422131
    np.mean(matches): 0.4069303467671706
ic| <ipython-input-13-6f9f2c390751>:98 in para_dataset()
    np.mean(scores): 0.38339704166731337
    np.mean(matches): 0.4059081514341886
ic| <ipython-input-13-6f9f2c390751>:98 in para_dataset()
    np.mean(scores): 0.3679153816086173
    np.mean(matches): 0.361741093751438
ic| <ipython-input-13-6f9f2c390751>:98 in para_dataset()
    np.mean(scores): 0.37094642577992554
    np.mean(matches): 0.386000169924645


In [16]:
dtrain = pd.concat([ds[1], ds[2], ds[3], ds[4]])

In [17]:
dvalid = ds[0]

In [50]:
# dtrain = d[d.subfold != 0]
# # dtrain = d0[d0.subfold == 1]
# dvalid = d[d.subfold == 0]
reg_cols =  [
              'num_words', 'start', 'start_ratio', 'end', 'end_ratio',
              'start_sep_prob', 'end_sep_prob', 'para_len', 'para_len_ratio',
              'num_classes', 
              'mean_prob', 
              # 'max_prob',
              # 'start_prob', 'end_prob', 
              # 'seps', 'sep_ratio', 
              'class_ratio',
              # 'sep_add_prob', 'sep_mul_prob', 
              # 'is_top_class', 'is_top_class2',
            #   'class_max_prob', 'class_min_prob'
        ]
cat_cols = [
            'cls', 
            # 'start_cls', 'end_cls',
            # 'top_class',
            # 'top_class2',
            'pre_cls', 
            'next_cls'
            ]
label_col = 'match'
label_col = 'score'
cols = reg_cols + cat_cols
X_train = dtrain[cols]
y_train = dtrain[[label_col]]

In [51]:
X_valid = dvalid[cols]
# y_valid = dvalid[[label_col]]
y_valid = dvalid[['match']]

In [49]:
learning_rate = 0.1
num_boost_round = 1000
params = {
          # "objective": "binary",
          "objective": "regression" if label_col is 'score' else 'binary',
          # "objective": "cross_entropy",
          "metric": "auc",
          "boosting_type": "gbdt",
          "learning_rate": learning_rate,
          "num_leaves": 12,
          "max_bin": 256,
          "feature_fraction": 0.75,
          "verbosity": 0,
          # "drop_rate": 0.1,
          "is_unbalance": True,
          # "max_drop": 50,
          "min_child_samples": 20,
          "min_child_weight": 150,
          "min_split_gain": 0,
          "bagging_freq": 5,
          "bagging_fraction": 0.9,
          "num_trees": 100,
          "subsample": 0.9
          }

In [52]:
d_train = lgb.Dataset(X_train, y_train)
d_valid = lgb.Dataset(X_valid, y_valid, reference=d_train)
bst = lgb.train(params, d_train, num_boost_round, valid_sets=d_valid, 
                categorical_feature=cat_cols,
                verbose_eval=1,
                early_stopping_rounds=10)

[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=0.9 will be ignored. Current value: bagging_fraction=0.9
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=0.9 will be ignored. Current value: bagging_fraction=0.9
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.256642 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=0.9 will be ignored. Current value: bagging_fraction=0.9
[1]	valid_0's auc: 0.694732
Training until validation scores don't improve for 10 rounds
[2]	valid_0's auc: 0.7126
[3]	valid_0's auc: 0.722943
[4]	valid_0's auc: 0.721988
[5]	valid_0's auc: 0.722778
[6]	valid_0's auc: 0.726869
[7]	valid_0's auc: 0.73475
[8]	valid_0's auc: 0.737985
[9]	valid_0's auc: 0.738757
[10]	valid_0's auc: 0.739141
[11]	valid_0's auc: 0.739041
[12]	valid_0's auc: 0.739742
[13]	valid_0's auc: 0.74043
[14]	valid_0's auc: 0.741195
[15]	valid_0's auc: 0.740814
[

In [53]:
l = list(zip(bst.feature_name(), bst.feature_importance()))
l.sort(key=lambda x: -x[1])
l

[('cls', 138),
 ('next_cls', 117),
 ('end_sep_prob', 105),
 ('num_words', 97),
 ('mean_prob', 92),
 ('start_sep_prob', 88),
 ('para_len_ratio', 76),
 ('pre_cls', 66),
 ('para_len', 58),
 ('num_classes', 55),
 ('end_ratio', 52),
 ('start_ratio', 45),
 ('class_ratio', 45),
 ('start', 34),
 ('end', 32)]

In [54]:
def greedy_decodes(df, cols, para_classifier):
  proba_thresh = {
    "Lead": 0.99,
    "Position": 0.55,
    "Evidence": 0.7,
    "Claim": 0.55,
    "Concluding Statement": 0.99,
    "Counterclaim": 0.5,
    "Rebuttal": 0.5,
    "Nothing": 0.1,
  }

  df['pred'] = para_classifier.predict(df[cols])
  ids = set(df.id)
  ids_list, types_list, preds_list = [], [], []
  for id in tqdm(ids):
    d = df[df.id == id]
    d = d.sort_values(['pred'], ascending=[False])
    num_words = d.num_words.values[0]
    used = np.zeros(num_words)
    for row in d.itertuples():
      start, end = row.start, row.end
      cls = row.cls
      if used[start: end].sum() == 0:
        # used[start: end] = 1
        # if cls > 0:
        if row.mean_prob > proba_thresh[id2dis[cls]]:
          used[start: end] = 1
          if cls > 0:
            ids_list.append(id)
            types_list.append(id2dis[cls])
            preds_list.append(' '.join([str(x) for x in range(start, end)]))
      if used.sum() == num_words:
        break
          
  df = pd.DataFrame({
    'id': ids_list,
    'class': types_list,
    'predictionstring': preds_list,
  })
  return df

In [55]:
df_pred = greedy_decodes(dvalid, cols, bst)

  0%|          | 0/3136 [00:00<?, ?it/s]

In [56]:
dvalid['pred']

0        0.65
1        0.36
2        0.29
3        0.29
4        0.29
         ... 
306013   0.47
306014   0.48
306015   0.47
306016   0.47
306017   0.48
Name: pred, Length: 306018, dtype: float64

In [57]:
df_gt = pd.read_csv('../working/offline/44/valid_gt.csv')

In [58]:
res = calc_metrics(df_gt[df_gt.id.isin(set(df_pred.id))], df_pred)
ic(res['f1/Overall'])
ic(res)

calc_f1:   0%|          | 0/7 [00:00<?, ?it/s]

ic| <ipython-input-58-cb78f7a414c1>:2 in <module>
    res['f1/Overall']: 0.7026877556982097
ic| <ipython-input-58-cb78f7a414c1>:3 in <module>
    res: {'acc/Claim': 0.7124468451902081,
          'acc/Concluding Statement': 0.8628015844436442,
          'acc/Counterclaim': 0.630665380906461,
          'acc/Evidence': 0.7744429286039108,
          'acc/Lead': 0.8307528029898559,
          'acc/Position': 0.7401282483968951,
          'acc/Rebuttal': 0.515625,
          'cm/Claim': array([[6199, 2502],
                [3953, 4748]]),
          'cm/Concluding Statement': array([[2396,  381],
                [ 329, 2448]]),
          'cm/Counterclaim': array([[654, 383],
                [526, 511]]),
          'cm/Evidence': array([[6812, 1984],
                [2406, 6390]]),
          'cm/Lead': array([[1556,  317],
                [ 300, 1573]]),
          'cm/Position': array([[2193,  770],
                [ 916, 2047]]),
          'cm/Rebuttal': array([[396, 372],
                [462,

{'f1/Overall': 0.7026877556982097,
 'f1/Claim': 0.6576141728106932,
 'acc/Claim': 0.7124468451902081,
 'recall/Claim': 0.610618597320725,
 'cm/Claim': array([[6199, 2502],
        [3953, 4748]]),
 'f1/Evidence': 0.7563006550460752,
 'acc/Evidence': 0.7744429286039108,
 'recall/Evidence': 0.738988934692992,
 'cm/Evidence': array([[6812, 1984],
        [2406, 6390]]),
 'f1/Position': 0.7223320158102767,
 'acc/Position': 0.7401282483968951,
 'recall/Position': 0.7053715020907044,
 'cm/Position': array([[2193,  770],
        [ 916, 2047]]),
 'f1/Concluding Statement': 0.8709560159941839,
 'acc/Concluding Statement': 0.8628015844436442,
 'recall/Concluding Statement': 0.8792660550458715,
 'cm/Concluding Statement': array([[2396,  381],
        [ 329, 2448]]),
 'f1/Lead': 0.8345400911772594,
 'acc/Lead': 0.8307528029898559,
 'recall/Lead': 0.8383620689655172,
 'cm/Lead': array([[1556,  317],
        [ 300, 1573]]),
 'f1/Counterclaim': 0.5899864682002707,
 'acc/Counterclaim': 0.63066538090646

In [59]:
# df_pred2[df_pred2.id=='4125C404E34B']

In [ ]:
# dvalid[dvalid.id=='4125C404E34B'][['id', 'pred', 'match', 'score', 'start', 'end', 'cls', 'mean_prob', 'start_sep_prob', 'end_sep_prob']]

In [30]:
# gts['4125C404E34B']

In [31]:
# ALL_CLASSES

In [32]:
# df_pred2 = df_pred2[df_pred2.id.isin(set(df_pred.id))]

In [33]:
# df_pred_fold1 = greedy_decodes(ds[1], cols, bst)

In [34]:
# res = calc_metrics(df_gt[df_gt.id.isin(set(df_pred_fold1.id))], df_pred_fold1)
# ic(res)

In [35]:
valid_ids = set(dvalid.id)

In [36]:
len(valid_ids)

3136

In [37]:
from src.config import proba_thresh

In [38]:
def get_pred_bystart(x, post_adjust=True, pred_info=None):
  MIN_LEN = FLAGS.para_min_len  #2
  MIN_LEN2 = FLAGS.para_min_len2 #6
  NUM_CLASSES = len(id2dis)
  pred = x['preds']
  total = len(pred)
  # by prob not logit
  probs = x['probs'] 
  # probs = x['pred']
  start_prob = x['start_probs'] if 'start_probs' in x else None
  # ic((start_prob[:,1] > start_prob[:,0]).astype(int).sum())
  pre_type = None
  # predictionstring list
  preds_list = []
  # store each pred word_id for one precitionstring
  preds = [] 
  pre_scores = np.zeros_like(probs[0])
  
  types = []
    
  pre_probs = None
  for i in range(total):    
    if pre_scores.shape[-1] > NUM_CLASSES:
      pre_type = id2dis[int(np.argmax(pre_scores) / 2)]
    else:
      pre_type = id2dis[np.argmax(pre_scores)]
    
    if pre_scores.shape[-1] > NUM_CLASSES:
      now_type = id2dis[int(np.argmax(probs[i]) / 2)]
    else:
      now_type = id2dis[np.argmax(probs[i])]
    
    is_sep = False
    if start_prob[i].sum() == 0:
      is_sep = True
      
    if start_prob is None:
      is_sep = pred[i] % 2 == 1
    else:
      is_sep = start_prob[i][1] > 0.5
      
      if post_adjust:
        if i > 0:
          # 注意目前最高线上版本依然是按照第一个model取pred而不是ensemble的结果取pred信息，另外有adjacent rule待验证
          if pred[i] != pred[i - 1]:
            if start_prob[i][1] > 0.3 and (pre_type in  ['Rebuttal', 'Counterclaim']):
                is_sep = True
            # 这个目前有利于shortformer 不利于longformer longformer的start预测更准确
            # 之前小模型集成这个是707 708有1k收益 但是目前看小模型自己集成之后似乎也可以去掉
            if FLAGS.adjacent_rule:
              if pre_probs[pred[i - 1]] > 0.5 and probs[i][pred[i]] > 0.5 and len(preds) >= min_thresh[id2dis[pred[i - 1]]]:
                is_sep = True
          else:
            if pre_type == 'Evidence':
              if start_prob[i][1] < 0.55:
                is_sep = False
                
          if start_prob[i][1] > 0.1:
            start = i - len(preds)
            end = i
            try:
              pred_value = pred_info[(pred_info.start==start) & (pred_info.end==end) & (pred_info.cls==pre_type)]['pred'].values[0]
            except Exception:
              pred_value = 0
            # print(start_prob[i][1], pred_value)
            if pred_value > 0.8:
              is_sep = True
              
      # else:
      #   if pred[i] != pred[i - 1]:
      #     is_sep = True
            
    if is_sep:
      if preds:  
        if pre_type != 'Nothing':
          if post_adjust:
            if len(preds) < MIN_LEN:
              # 低置信度的干脆放弃召回 更安全 pass not continue
              pass
            else:
              if pre_probs.max() > proba_thresh[pre_type]:
                preds_list.append(' '.join(preds))
                types.append(pre_type)
          else:
            preds_list.append(' '.join(preds))
            types.append(pre_type)
            
        preds = []
        pre_scores = np.zeros_like(probs[0])
              
    pre_scores += probs[i] 
    pre_probs = gezi.softmax(pre_scores)
    preds.append(str(i))
    
  if preds:
    if pre_scores.shape[-1] > NUM_CLASSES:
      pre_type = id2dis[int(np.argmax(pre_scores) / 2)]
    else:
      pre_type = id2dis[np.argmax(pre_scores)]
      
    # 结尾应该更长
    if pre_type != 'Nothing':
      if post_adjust:
        if len(preds) >= MIN_LEN2:
          if pre_probs.max() > proba_thresh[pre_type]:
            preds_list.append(' '.join(preds))
            types.append(pre_type)
      else:
        preds_list.append(' '.join(preds))
        types.append(pre_type)
  return types, preds_list


In [39]:
def get_preds_(x, post_adjust=True, selected_ids=None, fold=None, folds=5):  
  # ic(post_adjust)
  pred_fn = None
  if FLAGS.token2word:
    if FLAGS.pred_method == 'end':
      pred_fn = get_pred_byend
    elif FLAGS.pred_method == 'se':
      pred_fn = get_pred_byse
    elif FLAGS.pred_method == 'start':
      pred_fn = get_pred_bystart
    else:
      raise ValueError(FLAGS.pred_method)
  else:
    if FLAGS.pred_method == 'end':
      pred_fn = get_pred_byend2
    elif FLAGS.pred_method == 'se':
      pred_fn = get_pred_byse2
    elif FLAGS.pred_method == 'start':
      pred_fn = get_pred_bystart2
    else:
      raise ValueError(FLAGS.pred_method)
  # ic(pred_fn)

  total = len(x['id'])
  # with gezi.Timer('get_preds'):
  # ic(FLAGS.openmp)
  ids_list, types_list, preds_list = [], [], []
  for i in tqdm(range(total), desc='get_preds', leave=False):
    id = x['id'][i]
    if selected_ids is not None and id not in selected_ids:
      continue
    if fold is not None:
      if i % folds != fold:
        continue
    x_ = {}
    for key in x: 
      x_[key] = x[key][i]
    types, preds = pred_fn(x_, post_adjust=post_adjust, pred_info=dvalid[dvalid.id==id])
    ids_list.extend([id] * len(types))
    types_list.extend(types)
    preds_list.extend(preds)
 
  m = {
    'id': ids_list,
    'class': types_list,
    'predictionstring': preds_list
  }

  df = pd.DataFrame(m)
    
  return df

def get_preds(x, post_adjust=True, selected_ids=None, folds=5):  
  dfs = Manager().dict()
  with pymp.Parallel(folds) as p:
    for i in p.range(folds):
      dfs[i] = get_preds_(x, post_adjust, fold=i, folds=folds)
  return pd.concat(dfs.values())

In [40]:
gezi.init_flags()

In [41]:
df_pred2 = get_preds(info, selected_ids=valid_ids)

get_preds:   0%|          | 0/15594 [00:00<?, ?it/s]

In [42]:
len(df_pred2)

141294

In [43]:
len(set(df_pred2.id))

15592

In [44]:
res = calc_metrics(df_gt[df_gt.id.isin(valid_ids)], df_pred2[df_pred2.id.isin(valid_ids)])
res

calc_f1:   0%|          | 0/7 [00:00<?, ?it/s]

{'f1/Overall': 0.7112045216958619,
 'f1/Claim': 0.6738111185532485,
 'acc/Claim': 0.7063843577832991,
 'recall/Claim': 0.6441095350669819,
 'cm/Claim': array([[6539, 2718],
        [3613, 5644]]),
 'f1/Evidence': 0.7701508620689655,
 'acc/Evidence': 0.765121507333262,
 'recall/Evidence': 0.7752467729688687,
 'cm/Evidence': array([[7147, 2194],
        [2072, 7269]]),
 'f1/Position': 0.7319437712069802,
 'acc/Position': 0.7353896103896104,
 'recall/Position': 0.7285300739787713,
 'cm/Position': array([[2265,  815],
        [ 844, 2236]]),
 'f1/Concluding Statement': 0.8777757717999639,
 'acc/Concluding Statement': 0.8638948116560057,
 'recall/Concluding Statement': 0.8921100917431193,
 'cm/Concluding Statement': array([[2431,  383],
        [ 294, 2520]]),
 'f1/Lead': 0.8333779085316929,
 'acc/Lead': 0.8274030801911842,
 'recall/Lead': 0.8394396551724138,
 'cm/Lead': array([[1558,  325],
        [ 298, 1585]]),
 'f1/Counterclaim': 0.5938053097345133,
 'acc/Counterclaim': 0.6212962962962

In [45]:
res = calc_metrics(df_gt[df_gt.id.isin(valid_ids)], link_evidence(df_pred2[df_pred2.id.isin(valid_ids)]))
res

link_evidence:   0%|          | 0/3136 [00:00<?, ?it/s]

calc_f1:   0%|          | 0/7 [00:00<?, ?it/s]

{'f1/Overall': 0.7118046429163432,
 'f1/Claim': 0.6738111185532485,
 'acc/Claim': 0.7063843577832991,
 'recall/Claim': 0.6441095350669819,
 'cm/Claim': array([[6539, 2718],
        [3613, 5644]]),
 'f1/Evidence': 0.7743517106123339,
 'acc/Evidence': 0.7778264200503447,
 'recall/Evidence': 0.7709079075821673,
 'cm/Evidence': array([[7107, 2030],
        [2112, 7025]]),
 'f1/Position': 0.7319437712069802,
 'acc/Position': 0.7353896103896104,
 'recall/Position': 0.7285300739787713,
 'cm/Position': array([[2265,  815],
        [ 844, 2236]]),
 'f1/Concluding Statement': 0.8777757717999639,
 'acc/Concluding Statement': 0.8638948116560057,
 'recall/Concluding Statement': 0.8921100917431193,
 'cm/Concluding Statement': array([[2431,  383],
        [ 294, 2520]]),
 'f1/Lead': 0.8333779085316929,
 'acc/Lead': 0.8274030801911842,
 'recall/Lead': 0.8394396551724138,
 'cm/Lead': array([[1558,  325],
        [ 298, 1585]]),
 'f1/Counterclaim': 0.5938053097345133,
 'acc/Counterclaim': 0.621296296296

In [46]:
df_valid2 = pd.read_feather('../working/offline/44/1/base.bart.start/valid_gt.fea')

In [47]:
len(set(df_valid2.id))

3050

In [48]:
res = calc_metrics(df_valid2, link_evidence(df_pred2[df_pred2.id.isin(valid_ids)]))
res

link_evidence:   0%|          | 0/3136 [00:00<?, ?it/s]

calc_f1:   0%|          | 0/7 [00:00<?, ?it/s]

{'f1/Overall': 0.0,
 'f1/Claim': 0.0,
 'acc/Claim': 0.0,
 'recall/Claim': 0.0,
 'cm/Claim': array([[   0, 9257],
        [9814, -557]]),
 'f1/Evidence': 0.0,
 'acc/Evidence': 0.0,
 'recall/Evidence': 0.0,
 'cm/Evidence': array([[   0, 9135],
        [8981,  154]]),
 'f1/Position': 0.0,
 'acc/Position': 0.0,
 'recall/Position': 0.0,
 'cm/Position': array([[   0, 3080],
        [3018,   62]]),
 'f1/Concluding Statement': 0.0,
 'acc/Concluding Statement': 0.0,
 'recall/Concluding Statement': 0.0,
 'cm/Concluding Statement': array([[   0, 2814],
        [2633,  181]]),
 'f1/Lead': 0.0,
 'acc/Lead': 0.0,
 'recall/Lead': 0.0,
 'cm/Lead': array([[   0, 1883],
        [1823,   60]]),
 'f1/Counterclaim': 0.0,
 'acc/Counterclaim': 0.0,
 'recall/Counterclaim': 0.0,
 'cm/Counterclaim': array([[   0, 1080],
        [1161,  -81]]),
 'f1/Rebuttal': 0.0,
 'acc/Rebuttal': 0.0,
 'recall/Rebuttal': 0.0,
 'cm/Rebuttal': array([[  0, 786],
        [877, -91]])}